# Project Assignment 3: Mustaeen Ahmed

In [ ]:
import pygame
import moderngl
import numpy as np
import glm
from loadModelUsingAssimp_V3 import create3DAssimpObject

In [ ]:
width = 840
height = 480

pygame.init()
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLEBUFFERS, 1) 
pygame.display.gl_set_attribute(pygame.GL_MULTISAMPLESAMPLES, 16)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE) 
pygame.display.gl_set_attribute(pygame.GL_STENCIL_SIZE, 8)
pygame.display.set_mode((width, height), flags= pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title = "Project Assignment 03: Mustaeen Ahmed")

gl = moderngl.get_context()
FB = gl.detect_framebuffer()
gl.info["GL_VERSION"]

'4.6.0 NVIDIA 555.99'

In [68]:
model_file = "mario_obj/scene.gltf"
model_obj = create3DAssimpObject(model_file, verbose=False, textureFlag=True, normalFlag=True)
bound = model_obj.bound

_minP = bound.boundingBox[0]
_maxP = glm.vec3(bound.boundingBox[1].x, _minP.y, bound.boundingBox[1].z)
_center = (_minP + _maxP) / 2

plane_point = _center
plane_normal = glm.vec3(0, 1, 0)

square_quad_side = 3 * bound.radius
half_side_size = square_quad_side / 2

base_quad_geom_buffer = gl.buffer(np.array([
    _center.x - half_side_size, _center.y, _center.z - half_side_size, 0, 1, 0, 0, 0,
    _center.x + half_side_size, _center.y, _center.z - half_side_size, 0, 1, 0, 1, 0,
    _center.x + half_side_size, _center.y, _center.z + half_side_size, 0, 1, 0, 1, 1,
    _center.x - half_side_size, _center.y, _center.z + half_side_size, 0, 1, 0, 0, 1
]).astype("float32"))

_index = np.array([
    0, 1, 2,
    2, 3, 0
]).astype("int32")

base_quad_index_buffer = gl.buffer(_index)

_wood_texture_img = pygame.image.load("tile-squares-texture.jpg")
_texture_data = pygame.image.tobytes(_wood_texture_img, "RGB", True)
_texture = gl.texture(_wood_texture_img.get_size(), data=_texture_data, components=3)
_texture.build_mipmaps()

floor_sampler = gl.sampler(texture=_texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x=True, repeat_y=True)

Accumulates Normals in the geomDataList
Accumulates Textures in the geomDataList


In [69]:
def query_program(program):
    for name in program:
        member = program[name]
        print(name, type(member), member)


vertex_shader_code = '''
    #version 460 core
    layout (location=0) in vec3 position;
    layout (location=1) in vec3 normal;
    layout (location=2) in vec2 uv;

    uniform mat4 model, view, perspective;

    uniform mat4 lightViewMatrix;
    uniform mat4 lightProjectionMatrix;

    out vec2 f_uv;
    out vec3 f_normal;
    out vec3 f_position;
    out vec4 f_lightSpace;

    void main() {
        vec4 P = model * vec4(position, 1.0);
        f_position = P.xyz;

        f_uv = uv;
        mat3 normalMatrix = mat3(transpose(inverse(model)));
        f_normal = normalize(normalMatrix * normal);

        gl_Position = perspective * view * P;
        f_lightSpace = lightProjectionMatrix * lightViewMatrix * P;
    }
'''

fragment_shader_code = '''
    #version 430 core
    in vec2 f_uv;
    in vec3 f_normal;
    in vec3 f_position;
    in vec4 f_lightSpace;

    uniform sampler2D map;

    uniform sampler2D shadowMap;
    uniform bool biasFlag;
    uniform int pcf;
    uniform bool useShadowMap;

    uniform vec3 light;

    uniform float shininess;
    uniform vec3 eye_position;
    uniform vec3 k_diffuse;

    const vec3 up = vec3(0, 1, 0);
    const vec3 groundColor = vec3(0.3215686274509804, 0.4, 0.10980392156862745);
    const vec3 skyColor = vec3(0.7176470588235294, 0.7411764705882353, 0.7529411764705882);

    out vec4 out_color;

    float ComputeVisibilityFactor() {
        if (!useShadowMap) {
            return 1.0;
        }

        vec3 P_LightSpace_3D = f_lightSpace.xyz / f_lightSpace.w;

        float currentDepth = P_LightSpace_3D.z * 0.5 + 0.5;
        vec2 shadowUV = P_LightSpace_3D.xy * 0.5 + 0.5;

        if (shadowUV.x < 0.0 || shadowUV.x > 1.0 || shadowUV.y < 0.0 || shadowUV.y > 1.0) {
            return 1.0;
        }

        float bias = 0.0;
        if (biasFlag) {
            vec3 L = normalize(light - f_position);
            float cosTheta = max(dot(normalize(f_normal), L), 0.0);
            bias = max(0.0005 * (1.0 - cosTheta), 0.0005);
        }

        int kernelRadius = 0;
        if (pcf == 1) {
            kernelRadius = 1;
        } else if (pcf == 2) {
            kernelRadius = 2;
        } else if (pcf == 3) {
            kernelRadius = 3;
        }

        if (pcf == 0) {
            float storedDepth = texture(shadowMap, shadowUV).r;
            float adjustedDepth = currentDepth - bias;
            return (storedDepth < adjustedDepth) ? 0.0 : 1.0;
        }

        vec2 texelSize = 1.0 / textureSize(shadowMap, 0);
        float visibility = 0.0;
        int samples = 0;
        float adjustedDepth = currentDepth - bias;

        for (int x = -kernelRadius; x <= kernelRadius; ++x) {
            for (int y = -kernelRadius; y <= kernelRadius; ++y) {
                vec2 offset = vec2(x, y) * texelSize;
                float storedDepth = texture(shadowMap, shadowUV + offset).r;
                if (storedDepth >= adjustedDepth) {
                    visibility += 1.0;
                }
                samples++;
            }
        }

        visibility /= float(samples);
        return visibility;
    }

    vec3 computeColor() {
        vec3 L = normalize(light.xyz - f_position);
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);
        float NdotL = dot(N, L);
        vec3 color = vec3(0.0);

        float w = dot(N, up);
        vec3 ambientColor = 0.25 * (w * skyColor + (1.0 - w) * groundColor) * materialColor;

        float fractionalLightVisibility = ComputeVisibilityFactor();

        if (NdotL > 0.0) {
            vec3 diffuselyReflectedColor = materialColor * NdotL;
            vec3 V = normalize(eye_position - f_position);
            vec3 H = normalize(L + V);
            vec3 specularlyReflectedColor = vec3(0.0);
            if (shininess > 0.0)
                specularlyReflectedColor = vec3(1.0) * pow(max(dot(N, H), 0.0), shininess);
            color = fractionalLightVisibility * (k_diffuse * diffuselyReflectedColor + specularlyReflectedColor);
        }
        color += ambientColor;
        return color;
    }

    void main() {
        out_color = vec4(computeColor(), 1.0);
    }
'''

In [70]:
program = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

query_program(program)

model_obj.createRenderableAndSampler(program)

floor_renderer = gl.vertex_array(
    program,
    [(base_quad_geom_buffer, "3f 3f 2f", "position", "normal", "uv")],
    base_quad_index_buffer,
    index_element_size=4
)

normal <class '_moderngl.Attribute'> <Attribute: 1>
position <class '_moderngl.Attribute'> <Attribute: 0>
uv <class '_moderngl.Attribute'> <Attribute: 2>
biasFlag <class '_moderngl.Uniform'> <Uniform: 0>
eye_position <class '_moderngl.Uniform'> <Uniform: 1>
k_diffuse <class '_moderngl.Uniform'> <Uniform: 2>
light <class '_moderngl.Uniform'> <Uniform: 3>
lightProjectionMatrix <class '_moderngl.Uniform'> <Uniform: 4>
lightViewMatrix <class '_moderngl.Uniform'> <Uniform: 5>
map <class '_moderngl.Uniform'> <Uniform: 6>
model <class '_moderngl.Uniform'> <Uniform: 7>
pcf <class '_moderngl.Uniform'> <Uniform: 8>
perspective <class '_moderngl.Uniform'> <Uniform: 9>
shadowMap <class '_moderngl.Uniform'> <Uniform: 10>
shininess <class '_moderngl.Uniform'> <Uniform: 11>
useShadowMap <class '_moderngl.Uniform'> <Uniform: 12>
view <class '_moderngl.Uniform'> <Uniform: 13>


In [71]:
SHADOW_SIZE = (2048, 2048)
SHADOW_TEX_UNIT = 1

depth_buffer = gl.depth_texture(SHADOW_SIZE)
depth_buffer.compare_func = "<"

shadow_map_sampler = gl.sampler(
    texture=depth_buffer,
    filter=(gl.LINEAR, gl.LINEAR),
    repeat_x=False,
    repeat_y=False
)

shadow_fbo = gl.framebuffer(
    depth_attachment=depth_buffer
)

In [72]:
shadow_map_vertex_shader = '''
    #version 460 core
    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;

    out vec2 f_uv;

    void main() {
        f_uv = uv;
        gl_Position = vec4(position, 1.0);
    }
'''

shadow_map_fragment_shader = '''
    #version 460 core
    in vec2 f_uv;
    uniform sampler2D shadowMap;
    out vec4 out_color;

    void main() {
        float d = texture(shadowMap, f_uv).r;
        out_color = vec4(vec3(d), 1.0);
    }
'''

shadow_map_program = gl.program(
    vertex_shader=shadow_map_vertex_shader,
    fragment_shader=shadow_map_fragment_shader
)

In [73]:
shadow_quad_data = np.array([
    -1.0, -1.0, 0.0, 0.0, 0.0,
     1.0, -1.0, 0.0, 1.0, 0.0,
     1.0,  1.0, 0.0, 1.0, 1.0,
    -1.0,  1.0, 0.0, 0.0, 1.0, 
], dtype="f4")

shadow_quad_indices = np.array([0, 1, 2, 2, 3, 0], dtype="i4")

shadow_quad_buffer = gl.buffer(shadow_quad_data)
shadow_quad_index_buffer = gl.buffer(shadow_quad_indices)

In [74]:
shadow_quad_vao = gl.vertex_array(
    shadow_map_program,
    [(shadow_quad_buffer, "3f 2f", "position", "uv")],
    shadow_quad_index_buffer,
    index_element_size=4
)

In [75]:
def render_model():
    model_obj.render()

def render_floor():
    floor_sampler.use(0)
    program["model"].write(glm.mat4(1))
    floor_renderer.render()

def render_scene(view_matrix, perspective_matrix, light, eye):
    program["view"].write(view_matrix)
    program["perspective"].write(perspective_matrix)
    program["eye_position"].write(eye)
    program["light"].write(light)
    render_model()
    program["shininess"] = 0
    render_floor()

In [76]:
def show_shadow_map():
    size = width // 4
    gl.viewport = (width - size, height - size, size, size)
    gl.clear(0.5, 0.0, 0.5, viewport=gl.viewport)
    shadow_map_sampler.use(SHADOW_TEX_UNIT)
    shadow_map_program["shadowMap"] = SHADOW_TEX_UNIT
    shadow_quad_vao.render()
    gl.viewport = (0, 0, width, height)

In [77]:
displacement_vector = 4 * bound.radius * glm.rotateX(glm.vec3(0, 1, 0), glm.radians(85))
light_displacement_vector = 4 * bound.radius * glm.rotateZ(glm.vec3(0, 1, 0), glm.radians(45))

target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0, 1, 0)

fov_radian = glm.radians(30)
aspect = width / height
near = bound.radius
far = 20 * bound.radius
perspectiveMatrix = glm.perspective(fov_radian, aspect, near, far)

In [78]:
running = True
clock = pygame.time.Clock()
alpha = 0
lightAngle = 0
pcf = 0

pause = True
debug = False
bias = True
print(" Camera Orbiting Paused. No Shadow. Point Light. Bias. PCF : ", pcf)
gl.enable(gl.DEPTH_TEST)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_d:
                debug = not debug
            elif event.key == pygame.K_b:
                bias = not bias
            elif event.key == pygame.K_LEFT:
                lightAngle -= 5
            elif event.key == pygame.K_RIGHT:
                lightAngle += 5
            elif event.key == pygame.K_UP:
                if pcf < 3:
                    pcf += 1
                print("PCF level: ", pcf)
            elif event.key == pygame.K_DOWN:
                if pcf > 0:
                    pcf -= 1
                print("PCF level: ", pcf)
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspectiveMatrix = glm.perspective(fov_radian, width / height, near, far)
            gl.viewport = (0, 0, width, height)

    new_displacement_vector = glm.rotateY(displacement_vector, glm.radians(alpha))
    new_light_displacement_vector = glm.rotateY(light_displacement_vector, glm.radians(lightAngle))

    light_point = target_point + new_light_displacement_vector
    eye_point = target_point + new_displacement_vector

    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    lightViewMatrix = glm.lookAt(light_point, target_point, up_vector)
    lightPerspectiveMatrix = glm.perspective(glm.radians(60.0), 1.0, bound.radius, 10.0 * bound.radius)

    program["lightViewMatrix"].write(lightViewMatrix)
    program["lightProjectionMatrix"].write(lightPerspectiveMatrix)

    program["biasFlag"].value = bias
    program["pcf"].value = pcf

    shadow_fbo.use()
    gl.viewport = (0, 0, SHADOW_SIZE[0], SHADOW_SIZE[1])
    shadow_fbo.clear(depth=1.0)
    program["useShadowMap"].value = False

    render_scene(lightViewMatrix, lightPerspectiveMatrix, light_point, light_point)

    FB.use()
    gl.viewport = (0, 0, width, height)
    gl.clear(0.2, 0.2, 0.0)
    program["useShadowMap"].value = True
    shadow_map_sampler.use(SHADOW_TEX_UNIT)
    program["shadowMap"] = SHADOW_TEX_UNIT

    render_scene(viewMatrix, perspectiveMatrix, light_point, eye_point)

    if debug:
        show_shadow_map()

    pygame.display.flip()
    clock.tick(60)
    if not pause:
        alpha += 1
        if alpha > 360:
            alpha = 0

pygame.display.quit()


 Camera Orbiting Paused. No Shadow. Point Light. Bias. PCF :  0
